In [2]:
import os
import pandas as pd
from datetime import datetime, timedelta

def process_file(file_path):
    with open(file_path, 'r') as file:
        # Extracting t0 timestamp
        t0_str = ''
        for line in file:
            if line.startswith('#   t0:'):
                t0_str = line.split()[2]
                break
        
        if t0_str:
            t0 = datetime.fromisoformat(t0_str.replace('Z', '+00:00'))
            # Read and process the data after finding t0
            data = [line.split() for line in file if not line.startswith('#')]
        else:
            return pd.DataFrame()  # Return empty DataFrame if t0 is not found

    # Create a DataFrame
    df = pd.DataFrame(data, columns=['Rel Time [s]', 'Back Az [deg]', 'Tr. Velocity [m/s]', 'F-stat'])
    df = df.apply(pd.to_numeric)

    # Convert the first column to datetime
    df['Datetime'] = df['Rel Time [s]'].apply(lambda x: t0 + timedelta(seconds=x))
    df.drop('Rel Time [s]', axis=1, inplace=True)

    return df

# Define the folder path
folder_path = '/run/media/viblab/Markov2/Haykal/AnakKrakatauEWS/data/raw/I06AU/I06AU_FK_RESULTS'

# List all .DAT files in the folder
files = [f for f in os.listdir(folder_path) if f.endswith('fk_results.dat')]

# Process each file and concatenate
all_data = pd.DataFrame()
for file in files:
    file_path = os.path.join(folder_path, file)
    df = process_file(file_path)
    all_data = pd.concat([all_data, df], ignore_index=True)

# Save the combined DataFrame to a CSV file
output_csv_path = '/run/media/viblab/Markov2/Haykal/AnakKrakatauEWS/data/raw/I06AU/FK_Combined.csv'
all_data.to_csv(output_csv_path, index=False)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the CSV file
csv_file_path = '/run/media/viblab/Markov2/Haykal/AnakKrakatauEWS/data/raw/I06AU/FK_Combined.csv'  # Replace with your CSV file path
data = pd.read_csv(csv_file_path)

# Convert 'Datetime' from string to datetime object
data['Datetime'] = pd.to_datetime(data['Datetime'])

# Creating subplots
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))

# Plotting Trace Velocity
ax1.plot(data['Datetime'], data['Tr. Velocity [m/s]'], color='blue')
ax1.set_title('Trace Velocity over Time')
ax1.set_xlabel('Datetime')
ax1.set_ylabel('Trace Velocity [m/s]')

# Plotting Back Azimuth
ax2.plot(data['Datetime'], data['Back Az [deg]'], color='red')
ax2.set_title('Back Azimuth over Time')
ax2.set_xlabel('Datetime')
ax2.set_ylabel('Back Azimuth [deg]')

plt.tight_layout()
plt.show()
